### Next steps:

TODO:
1. Develop method for measuring wave propogation speed in image coordinates,  then find the flattened homography that makes this as constant as possible.
1. Create a pipeline that generates animations for each stage of the training process (normalization, clipping, learning), and compiles into a demo video.
1. Get tensorboard metrics logging working with this logger class: https://github.com/PyTorchLightning/pytorch-lightning/issues/1228#issuecomment-622963564


#### First get the basic Tensorflow Lightning training loop working

In [ ]:
import os
os.chdir('/home/erik/work/surfbreak/nbs')

In [ ]:
%load_ext autoreload
%autoreload 2
import torch
import pytorch_lightning as pl
from surfbreak.waveform_models import LitSirenNet

tb_logger = pl.loggers.TensorBoardLogger('logs/', name="wavefunc")

trainer = pl.Trainer(
    logger=tb_logger,
    limit_val_batches=1,
    max_epochs=5,
    gpus=1 if torch.cuda.is_available() else None,
    )

wavefunc_model = LitSirenNet(hidden_features=256, hidden_layers=3, first_omega_0=1.5, hidden_omega_0=11.7, squared_slowness=0.23,
                             steps_per_vid_chunk=100, learning_rate=1e-4, grad_loss_scale=0, wave_loss_scale=4.4e-8)

trainer.fit(wavefunc_model)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type  | Params | In sizes     | Out sizes                   
---------------------------------------------------------------------------------------
0 | model          | Siren | 198 K  | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]
1 | slowness_model | Siren | 8 K    | ?            | ?                           


1

In [ ]:
gradonly_model = LitSirenNet(hidden_features=256, hidden_layers=3, first_omega_0=1.5, hidden_omega_0=11.7, squared_slowness=0.23,
                             steps_per_vid_chunk=100, learning_rate=1e-4, grad_loss_scale=1e-4, wave_loss_scale=0)
trainer.fit(gradonly_model)

### Now get optuna optimization trials working

TODO:
1. Improve early stopping such that I have control over patience parameter and ensure the _minimum_ validation loss is passed to the optuna study.

The most well-regularized hyperparameters for simple (`mse_loss` only) ended up being 
256 hidden features, 3.7995 first_omega_0, 2.9312 hidden_omega_0

With wavefunc loss, `squared_slowness` of around 0.5 may be close

```
Finished trial#26 with value: 0.09781524538993835 with parameters: 
{'first_omega_0': 4.839289222946841, 'hidden_omega_0': 13.756932872278343, 'squared_slowness': 0.27488941275825124, 'wave_loss_scale': 9.252313787089657e-08}
```

In [ ]:
import os
import torch
import pytorch_lightning as pl
from surfbreak.waveform_models import LitSirenNet
from optuna.integration import PyTorchLightningPruningCallback
from surfbreak.studies import run_waveform_hyperparam_search, MetricsCallback
LOGDIR = 'logs'
MODELDIR = os.path.join(LOGDIR, 'opt_models')

def objective(trial):
    checkpoint_callback = pl.callbacks.ModelCheckpoint( # Filenames for each trial must be made unique
        os.path.join(MODELDIR, "trial_{}".format(trial.number), "{epoch}"), monitor="val_loss")
    tb_logger = pl.loggers.TensorBoardLogger(LOGDIR+'/', name="optuna")
    metrics_callback = MetricsCallback()     # Simple callback that saves metrics from each validation step.
    
    pl.seed_everything(42)
    trainer = pl.Trainer(logger=tb_logger, limit_val_batches=3,
                         max_epochs=20, 
                         gpus=1 if torch.cuda.is_available() else None,
                         callbacks=[metrics_callback],
                         #early_stop_callback=PyTorchLightningPruningCallback(trial, monitor="val_loss", patience=10),
                        )

    wavefunc_model = LitSirenNet(hidden_features=trial.suggest_categorical('hidden_features', [128, 256, 380]), #256,
                                 hidden_layers=trial.suggest_categorical('hidden_layers', [3]), #3,
                                 first_omega_0=trial.suggest_uniform('first_omega_0', 0.5, 2.5), #1.5, 
                                 hidden_omega_0=trial.suggest_uniform('hidden_omega_0', 8.5, 15), #11.7,
                                 squared_slowness=trial.suggest_uniform('squared_slowness', 0.20, 0.70), #0.23,
                                 steps_per_vid_chunk=100, 
                                 learning_rate=1e-4, grad_loss_scale=0, 
                                 wavefunc_loss_scale=trial.suggest_loguniform('wavefunc_loss_scale', 5e-9, 1e-7), #2e-8,
                                 wavespeed_loss_scale=trial.suggest_loguniform('wavespeed_loss_scale', 1e-4, 1e-2), #1e-3,
                                 xrange=(0,600), timerange=(0,3*30), chunk_duration=30, chunk_stride=30)
                                 # With default settings wavezone image dimensions are (y=139, x=1515)

    trainer.fit(wavefunc_model)
    return metrics_callback.metrics[-1]["val_loss"].item()


study = run_waveform_hyperparam_search(objective, n_trials=100, timeout=8*60*60, model_dir=MODELDIR, 
                                       prune=False, n_startup_trials=3, n_warmup_steps=5)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type  | Params | In sizes     | Out sizes                   
---------------------------------------------------------------------------------------
0 | model          | Siren | 436 K  | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]
1 | slowness_model | Siren | 8 K    | ?            | ?                           


[I 2020-07-04 23:49:03,414] Finished trial#0 with value: 0.13114261627197266 with parameters: {'hidden_features': 380, 'hidden_layers': 3, 'first_omega_0': 2.235704528549165, 'hidden_omega_0': 10.007569568983525, 'squared_slowness': 0.341543868797459, 'wavefunc_loss_scale': 5.97776850767863e-09, 'wavespeed_loss_scale': 0.00010504655720688889}. Best is trial#0 with value: 0.13114261627197266.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type  | Params | In sizes     | Out sizes                   
---------------------------------------------------------------------------------------
0 | model          | Siren | 50 K   | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]
1 | slowness_model | Siren | 8 K    | ?            | ?                           


[I 2020-07-05 00:14:28,659] Finished trial#1 with value: 0.1699213683605194 with parameters: {'hidden_features': 128, 'hidden_layers': 3, 'first_omega_0': 2.3559258343131404, 'hidden_omega_0': 9.344969683024756, 'squared_slowness': 0.2676454863899606, 'wavefunc_loss_scale': 8.834905911906638e-08, 'wavespeed_loss_scale': 0.0001170275128288722}. Best is trial#0 with value: 0.13114261627197266.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type  | Params | In sizes     | Out sizes                   
---------------------------------------------------------------------------------------
0 | model          | Siren | 436 K  | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]
1 | slowness_model | Siren | 8 K    | ?            | ?                           


[I 2020-07-05 00:41:56,554] Finished trial#2 with value: 0.17236721515655518 with parameters: {'hidden_features': 380, 'hidden_layers': 3, 'first_omega_0': 2.129257612629444, 'hidden_omega_0': 9.399822821219848, 'squared_slowness': 0.6099784694651422, 'wavefunc_loss_scale': 5.440452810779574e-08, 'wavespeed_loss_scale': 0.003370872594454371}. Best is trial#0 with value: 0.13114261627197266.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type  | Params | In sizes     | Out sizes                   
---------------------------------------------------------------------------------------
0 | model          | Siren | 436 K  | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]
1 | slowness_model | Siren | 8 K    | ?            | ?                           


[I 2020-07-05 01:09:14,285] Finished trial#3 with value: 0.17920605838298798 with parameters: {'hidden_features': 380, 'hidden_layers': 3, 'first_omega_0': 1.7518728527410257, 'hidden_omega_0': 10.579978152623935, 'squared_slowness': 0.5703117485127444, 'wavefunc_loss_scale': 6.31926617842639e-08, 'wavespeed_loss_scale': 0.006320617237339562}. Best is trial#0 with value: 0.13114261627197266.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type  | Params | In sizes     | Out sizes                   
---------------------------------------------------------------------------------------
0 | model          | Siren | 436 K  | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]
1 | slowness_model | Siren | 8 K    | ?            | ?                           


...

[I 2020-07-05 06:19:26,506] Finished trial#15 with value: 0.14361882209777832 with parameters: {'hidden_features': 256, 'hidden_layers': 3, 'first_omega_0': 2.482652532911402, 'hidden_omega_0': 9.70288078602397, 'squared_slowness': 0.20498187157958886, 'wavefunc_loss_scale': 5.1148083633873706e-09, 'wavespeed_loss_scale': 0.00019473375647797973}. Best is trial#0 with value: 0.13114261627197266.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type  | Params | In sizes     | Out sizes                   
---------------------------------------------------------------------------------------
0 | model          | Siren | 50 K   | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]
1 | slowness_model | Siren | 8 K    | ?            | ?                           


[I 2020-07-05 06:44:19,426] Finished trial#16 with value: 0.13705161213874817 with parameters: {'hidden_features': 128, 'hidden_layers': 3, 'first_omega_0': 1.5733094965641612, 'hidden_omega_0': 11.413516634536949, 'squared_slowness': 0.3011432851406898, 'wavefunc_loss_scale': 8.719025559700538e-09, 'wavespeed_loss_scale': 0.0007506909751440314}. Best is trial#0 with value: 0.13114261627197266.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type  | Params | In sizes     | Out sizes                   
---------------------------------------------------------------------------------------
0 | model          | Siren | 198 K  | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]
1 | slowness_model | Siren | 8 K    | ?            | ?                           


[I 2020-07-05 07:09:02,986] Finished trial#17 with value: 0.15659941732883453 with parameters: {'hidden_features': 256, 'hidden_layers': 3, 'first_omega_0': 1.1805431982185892, 'hidden_omega_0': 8.53323088079927, 'squared_slowness': 0.4221433613960459, 'wavefunc_loss_scale': 2.9280117083392725e-08, 'wavespeed_loss_scale': 0.0013088360341166924}. Best is trial#0 with value: 0.13114261627197266.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type  | Params | In sizes     | Out sizes                   
---------------------------------------------------------------------------------------
0 | model          | Siren | 198 K  | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]
1 | slowness_model | Siren | 8 K    | ?            | ?                           


[I 2020-07-05 07:33:48,048] Finished trial#18 with value: 0.14451110363006592 with parameters: {'hidden_features': 256, 'hidden_layers': 3, 'first_omega_0': 2.259033433218354, 'hidden_omega_0': 10.316182332434492, 'squared_slowness': 0.23879697853681708, 'wavefunc_loss_scale': 1.4501322824256637e-08, 'wavespeed_loss_scale': 0.00035772455783758626}. Best is trial#0 with value: 0.13114261627197266.


Number of finished trials: 19
Best trial was #0:
  Value: 0.13114261627197266
  Params: 
    hidden_features: 380
    hidden_layers: 3
    first_omega_0: 2.235704528549165
    hidden_omega_0: 10.007569568983525
    squared_slowness: 0.341543868797459
    wavefunc_loss_scale: 5.97776850767863e-09
    wavespeed_loss_scale: 0.00010504655720688889


In [ ]:
study.best_params

{'hidden_features': 380,
 'hidden_layers': 3,
 'first_omega_0': 2.235704528549165,
 'hidden_omega_0': 10.007569568983525,
 'squared_slowness': 0.341543868797459,
 'wavefunc_loss_scale': 5.97776850767863e-09,
 'wavespeed_loss_scale': 0.00010504655720688889}

In [ ]:
sdf = study.trials_dataframe()
top_trials = sdf.sort_values(by='value')[:5]
top_trials

,number,value,datetime_start,datetime_complete,duration,params_first_omega_0,params_hidden_features,params_hidden_layers,params_hidden_omega_0,params_squared_slowness,params_wavefunc_loss_scale,params_wavespeed_loss_scale,state
0,0,0.131143,2020-07-04 23:21:25.800355,2020-07-04 23:49:03.413944,00:27:37.613589,2.235705,380,3,10.007570,0.341544,5.977769e-09,0.000105,COMPLETE
11,11,0.131585,2020-07-05 04:15:30.272110,2020-07-05 04:40:23.186604,00:24:52.914494,2.496738,256,3,10.969015,0.204919,5.471891e-09,0.000435,COMPLETE
10,10,0.132419,2020-07-05 03:50:36.195951,2020-07-05 04:15:30.269835,00:24:54.073884,2.469356,256,3,10.737842,0.202324,5.536735e-09,0.000381,COMPLETE
16,16,0.137052,2020-07-05 06:19:26.508430,2020-07-05 06:44:19.426469,00:24:52.918039,1.573309,128,3,11.413517,0.301143,8.719026e-09,0.000751,COMPLETE
14,14,0.137106,2020-07-05 05:29:46.862792,2020-07-05 05:54:38.817953,00:24:51.955161,2.275557,256,3,11.615780,0.688081,9.518309e-09,0.000260,COMPLETE


In [ ]:
print(top_trials.mean())
print(top_trials.std())

number                                            9.5
value                                        0.136229
duration                       0 days 00:25:27.666368
params_first_omega_0                          1.95588
params_hidden_features                            255
params_hidden_layers                                3
params_hidden_omega_0                         11.3476
params_squared_slowness                      0.400915
params_wavefunc_loss_scale                8.04706e-09
params_wavespeed_loss_scale               0.000499383
dtype: object
number                                        5.12696
value                                      0.00405406
duration                       0 days 00:01:14.189584
params_first_omega_0                         0.630964
params_hidden_features                         95.253
params_hidden_layers                                0
params_hidden_omega_0                         0.97345
params_squared_slowness                      0.179524
params_wavefun